## Aspect Term Extraction (ATE) Training and Fine Tuning for Large Language Models on German hospital reviews using the special OB-Tagging


In [1]:
import torch
import os

import spacy
import ast  # To safely evaluate strings as Python objects

from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from datasets import Dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import evaluate

from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

# We need the sys package to load modules from another directory:
import sys
sys.path.append('../')
from functions.ate_model_train_OB import *

/home/sc.uni-leipzig.de/ch31qoni/venv/testabsa/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

Is CUDA available: True
CUDA version: 12.6
GPU device name: NVIDIA A30


In [3]:
# Load the dataset into a DataFrame
data = pd.read_csv("./data/hospitalABSA/patient_review_labels_absa.csv")
data_ano = pd.read_csv("./data/hospitalABSA/patient_review_labels_absa_ano.csv")

In [4]:
models = ["google-bert/bert-base-german-cased","dbmdz/bert-base-german-cased", "dbmdz/bert-base-german-uncased",
          "FacebookAI/xlm-roberta-base", "TUM/GottBERT_base_best", "TUM/GottBERT_filtered_base_best", "TUM/GottBERT_base_last",
          "distilbert/distilbert-base-german-cased", "GerMedBERT/medbert-512", "deepset/gbert-base"]

### 1. Train standard ATE Models for 5, 6, 7, 8, 10, 12 epochs

In [5]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=5)
    print()
# GPU: NVIDIA GeForce RTX 2080 Ti

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 3653.54 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for google-bert/bert-base-german-cased with 5 epochs and random seeds: 42, 42



...

['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for deepset/gbert-base with 5 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.129026,0.808000,0.795276,0.801587
2,0.190300,0.153711,0.833333,0.826772,0.830040
3,0.095500,0.203675,0.836735,0.807087,0.821643
4,0.038200,0.209330,0.830769,0.850394,0.840467
5,0.023900,0.232320,0.825095,0.854331,0.839458


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 3005.47 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.91      0.79      0.84       315

   micro avg       0.91      0.79      0.84       315
   macro avg       0.91      0.79      0.84       315
weighted avg       0.91      0.79      0.84       315

Precision Score: 0.9084249084249084
Recall Score: 0.7873015873015873
F1 Score: 0.8435374149659863
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labe

In [11]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=5)
    print()
# GPU: NVIDIA A30

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 3816.95 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for google-bert/bert-base-german-cased with 5 epochs and random seeds: 42, 42



...

['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for deepset/gbert-base with 5 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.130743,0.802372,0.799213,0.800789
2,0.190700,0.153970,0.823529,0.826772,0.825147
3,0.094100,0.199375,0.825911,0.803150,0.814371
4,0.039700,0.215434,0.834646,0.834646,0.834646
5,0.023900,0.240411,0.829457,0.842520,0.835938


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5812.76 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.91      0.79      0.84       315

   micro avg       0.91      0.79      0.84       315
   macro avg       0.91      0.79      0.84       315
weighted avg       0.91      0.79      0.84       315

Precision Score: 0.9054545454545454
Recall Score: 0.7904761904761904
F1 Score: 0.8440677966101695
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labe

In [ ]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=6)
    print()
# GPU: NVIDIA GeForce RTX 2080 Ti

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 3269.87 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for google-bert/bert-base-german-cased with 6 epochs and random seeds: 42, 42



...

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Some weights of BertForTokenClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferen

mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 3444.10 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for deepset/gbert-base with 6 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.140835,0.832618,0.763780,0.796715
2,0.195200,0.145978,0.833333,0.866142,0.849421
3,0.094000,0.249574,0.889381,0.791339,0.837500
4,0.032800,0.247657,0.841897,0.838583,0.840237


In [7]:
ate_model(data, "deepset/gbert-base", rn1=42, rn2=42, epochs=6)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', '

mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 6037.98 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for deepset/gbert-base with 6 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.133192,0.844156,0.767717,0.804124
2,0.187100,0.138036,0.855372,0.814961,0.834677
3,0.090400,0.192427,0.877729,0.791339,0.832298
4,0.029500,0.215252,0.868085,0.803150,0.834356
5,0.019800,0.242905,0.851406,0.834646,0.842942
6,0.019800,0.251559,0.866667,0.818898,0.842105


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5502.28 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.88      0.77      0.82       315

   micro avg       0.88      0.77      0.82       315
   macro avg       0.88      0.77      0.82       315
weighted avg       0.88      0.77      0.82       315

Precision Score: 0.8836363636363637
Recall Score: 0.7714285714285715
F1 Score: 0.8237288135593221
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labe

In [12]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=6)
    print()
# GPU: NVIDIA A30

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 6472.20 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for google-bert/bert-base-german-cased with 6 epochs and random seeds: 42, 42



...

['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for deepset/gbert-base with 6 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.142978,0.830435,0.751969,0.789256
2,0.195200,0.146292,0.830827,0.870079,0.850000
3,0.094200,0.248587,0.870690,0.795276,0.831276
4,0.032800,0.230214,0.830040,0.826772,0.828402
5,0.020200,0.262754,0.820611,0.846457,0.833333
6,0.020200,0.296022,0.837398,0.811024,0.824000


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5696.20 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.84      0.82      0.83       315

   micro avg       0.84      0.82      0.83       315
   macro avg       0.84      0.82      0.83       315
weighted avg       0.84      0.82      0.83       315

Precision Score: 0.8403908794788274
Recall Score: 0.819047619047619
F1 Score: 0.8295819935691318
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Label

In [8]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=7)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 5357.59 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for google-bert/bert-base-german-cased with 7 epochs and random seeds: 42, 42



...

['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for deepset/gbert-base with 7 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.120722,0.821429,0.814961,0.818182
2,0.190500,0.147927,0.844828,0.771654,0.806584
3,0.089500,0.235073,0.865801,0.787402,0.824742
4,0.032900,0.243635,0.838057,0.814961,0.826347
5,0.020200,0.280862,0.815789,0.854331,0.834615
6,0.020200,0.318454,0.808989,0.850394,0.829175
7,0.008000,0.329083,0.812500,0.818898,0.815686


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5193.43 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.88      0.82      0.85       315

   micro avg       0.88      0.82      0.85       315
   macro avg       0.88      0.82      0.85       315
weighted avg       0.88      0.82      0.85       315

Precision Score: 0.8831615120274914
Recall Score: 0.8158730158730159
F1 Score: 0.8481848184818481
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labe

In [9]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=8)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 6466.76 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for google-bert/bert-base-german-cased with 8 epochs and random seeds: 42, 42



...

['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for deepset/gbert-base with 8 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.135569,0.827731,0.775591,0.800813
2,0.192400,0.144324,0.863636,0.822835,0.842742
3,0.087100,0.244297,0.852941,0.799213,0.825203
4,0.031700,0.238428,0.850806,0.830709,0.840637
5,0.020400,0.267196,0.852000,0.838583,0.845238
6,0.020400,0.314748,0.842520,0.842520,0.842520
7,0.007800,0.332497,0.855967,0.818898,0.837022
8,0.002500,0.328087,0.854839,0.834646,0.844622


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5191.10 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.92      0.80      0.85       315

   micro avg       0.92      0.80      0.85       315
   macro avg       0.92      0.80      0.85       315
weighted avg       0.92      0.80      0.85       315

Precision Score: 0.9163636363636364
Recall Score: 0.8
F1 Score: 0.8542372881355933
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 

In [10]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=10)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 5363.29 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for google-bert/bert-base-german-cased with 10 epochs and random seeds: 42, 42



...

['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for deepset/gbert-base with 10 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.143593,0.812749,0.803150,0.807921
2,0.189900,0.182269,0.814229,0.811024,0.812623
3,0.088600,0.229563,0.868996,0.783465,0.824017
4,0.033300,0.266637,0.868085,0.803150,0.834356
5,0.020400,0.359470,0.858407,0.763780,0.808333
6,0.020400,0.353546,0.821012,0.830709,0.825832
7,0.008400,0.367762,0.865801,0.787402,0.824742
8,0.002700,0.382766,0.825397,0.818898,0.822134
9,0.000900,0.393986,0.830645,0.811024,0.820717
10,0.002400,0.402136,0.822835,0.822835,0.822835


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5893.07 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.90      0.77      0.83       315

   micro avg       0.90      0.77      0.83       315
   macro avg       0.90      0.77      0.83       315
weighted avg       0.90      0.77      0.83       315

Precision Score: 0.8992537313432836
Recall Score: 0.765079365079365
F1 Score: 0.8267581475128645
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Label

In [11]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=12)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 6214.97 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for google-bert/bert-base-german-cased with 12 epochs and random seeds: 42, 42



...

['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for deepset/gbert-base with 12 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.139483,0.859031,0.767717,0.810811
2,0.187000,0.161027,0.843137,0.846457,0.844794
3,0.090300,0.227668,0.844000,0.830709,0.837302
4,0.033200,0.249084,0.871681,0.775591,0.820833
5,0.021500,0.325955,0.822134,0.818898,0.820513
6,0.021500,0.334239,0.838843,0.799213,0.818548
7,0.009400,0.377150,0.832653,0.803150,0.817635
8,0.005100,0.371767,0.851406,0.834646,0.842942
9,0.001900,0.366353,0.854251,0.830709,0.842315
10,0.001500,0.371999,0.852590,0.842520,0.847525


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5920.31 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.90      0.83      0.86       315

   micro avg       0.90      0.83      0.86       315
   macro avg       0.90      0.83      0.86       315
weighted avg       0.90      0.83      0.86       315

Precision Score: 0.9
Recall Score: 0.8285714285714286
F1 Score: 0.8628099173553719
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 

### 2. Train category-aware ATE Models for 5, 6, 7, 8, 10, 12 epochs

In [5]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=5)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 5290.15 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1035: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-Krankenhaus' 'B-mediz. Service' 'B-Personal' 'B-anderer Service'
 'B-Arzt' 'B-Pflegepersonal' 'O']
{0: 5.983217355710193, 1: 4.959959280624364, 2: 18.317042606516292, 3: 10.546176046176047, 4: 6.8239962651727355, 5: 14.014381591562799, 6: 0.1596439493228484}
Training google-bert/bert-base-german-cased for 5 epochs with random seeds 42, 42



...

['B-Krankenhaus' 'B-mediz. Service' 'B-Personal' 'B-anderer Service'
 'B-Arzt' 'B-Pflegepersonal' 'O']
{0: 5.983217355710193, 1: 4.959959280624364, 2: 18.317042606516292, 3: 10.546176046176047, 4: 6.8239962651727355, 5: 14.014381591562799, 6: 0.1596439493228484}
Training deepset/gbert-base for 5 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.229855,0.685185,0.582677,0.629787
2,0.337800,0.188612,0.784689,0.645669,0.708423
3,0.147400,0.217004,0.758621,0.692913,0.724280
4,0.059300,0.246612,0.769565,0.696850,0.731405
5,0.038100,0.255069,0.755102,0.728346,0.741483


Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5930.81 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.88      0.95      0.91        55
    Krankenhaus       0.90      0.62      0.73       117
       Personal       0.80      0.62      0.70        13
 Pflegepersonal       1.00      0.94      0.97        18
anderer Service       0.69      0.51      0.59        35
 mediz. Service       0.88      0.73      0.79        77

      micro avg       0.87      0.71      0.78       315
      macro avg       0.86      0.73      0.78       315
   weighted avg       0.87      0.71      0.77       315

Precision Score: 0.87109375
Recall Score: 0.707936507936508
F1 Score: 0.7810858143607706
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', '

In [6]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=6)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 6282.81 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1035: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-Krankenhaus' 'B-mediz. Service' 'B-Personal' 'B-anderer Service'
 'B-Arzt' 'B-Pflegepersonal' 'O']
{0: 5.983217355710193, 1: 4.959959280624364, 2: 18.317042606516292, 3: 10.546176046176047, 4: 6.8239962651727355, 5: 14.014381591562799, 6: 0.1596439493228484}
Training google-bert/bert-base-german-cased for 6 epochs with random seeds 42, 42



...

['B-Krankenhaus' 'B-mediz. Service' 'B-Personal' 'B-anderer Service'
 'B-Arzt' 'B-Pflegepersonal' 'O']
{0: 5.983217355710193, 1: 4.959959280624364, 2: 18.317042606516292, 3: 10.546176046176047, 4: 6.8239962651727355, 5: 14.014381591562799, 6: 0.1596439493228484}
Training deepset/gbert-base for 6 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.238001,0.679070,0.574803,0.622601
2,0.330400,0.187735,0.740260,0.673228,0.705155
3,0.143500,0.212130,0.792531,0.751969,0.771717
4,0.056900,0.230783,0.768595,0.732283,0.750000
5,0.033400,0.254670,0.741176,0.744094,0.742633
6,0.033400,0.252418,0.767068,0.751969,0.759443


Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5821.54 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.85      0.95      0.90        55
    Krankenhaus       0.90      0.76      0.82       117
       Personal       0.69      0.69      0.69        13
 Pflegepersonal       0.94      0.94      0.94        18
anderer Service       0.80      0.34      0.48        35
 mediz. Service       0.85      0.74      0.79        77

      micro avg       0.86      0.75      0.80       315
      macro avg       0.84      0.74      0.77       315
   weighted avg       0.86      0.75      0.79       315

Precision Score: 0.8644688644688645
Recall Score: 0.7492063492063492
F1 Score: 0.8027210884353742
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [7]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=7)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 5989.15 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1035: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-Krankenhaus' 'B-mediz. Service' 'B-Personal' 'B-anderer Service'
 'B-Arzt' 'B-Pflegepersonal' 'O']
{0: 5.983217355710193, 1: 4.959959280624364, 2: 18.317042606516292, 3: 10.546176046176047, 4: 6.8239962651727355, 5: 14.014381591562799, 6: 0.1596439493228484}
Training google-bert/bert-base-german-cased for 7 epochs with random seeds 42, 42



...

['B-Krankenhaus' 'B-mediz. Service' 'B-Personal' 'B-anderer Service'
 'B-Arzt' 'B-Pflegepersonal' 'O']
{0: 5.983217355710193, 1: 4.959959280624364, 2: 18.317042606516292, 3: 10.546176046176047, 4: 6.8239962651727355, 5: 14.014381591562799, 6: 0.1596439493228484}
Training deepset/gbert-base for 7 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.238920,0.709845,0.539370,0.612975
2,0.327400,0.200221,0.766520,0.685039,0.723493
3,0.144800,0.232318,0.766234,0.696850,0.729897
4,0.057800,0.245373,0.750000,0.732283,0.741036
5,0.034800,0.260837,0.736059,0.779528,0.757170
6,0.034800,0.271196,0.746154,0.763780,0.754864
7,0.011000,0.281080,0.753846,0.771654,0.762646


Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5440.43 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.88      0.93      0.90        55
    Krankenhaus       0.90      0.72      0.80       117
       Personal       0.75      0.69      0.72        13
 Pflegepersonal       0.85      0.94      0.89        18
anderer Service       0.81      0.49      0.61        35
 mediz. Service       0.76      0.78      0.77        77

      micro avg       0.84      0.76      0.80       315
      macro avg       0.83      0.76      0.78       315
   weighted avg       0.84      0.76      0.79       315

Precision Score: 0.8409893992932862
Recall Score: 0.7555555555555555
F1 Score: 0.7959866220735786
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [8]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=8)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 6472.89 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1035: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-Krankenhaus' 'B-mediz. Service' 'B-Personal' 'B-anderer Service'
 'B-Arzt' 'B-Pflegepersonal' 'O']
{0: 5.983217355710193, 1: 4.959959280624364, 2: 18.317042606516292, 3: 10.546176046176047, 4: 6.8239962651727355, 5: 14.014381591562799, 6: 0.1596439493228484}
Training google-bert/bert-base-german-cased for 8 epochs with random seeds 42, 42



...

['B-Krankenhaus' 'B-mediz. Service' 'B-Personal' 'B-anderer Service'
 'B-Arzt' 'B-Pflegepersonal' 'O']
{0: 5.983217355710193, 1: 4.959959280624364, 2: 18.317042606516292, 3: 10.546176046176047, 4: 6.8239962651727355, 5: 14.014381591562799, 6: 0.1596439493228484}
Training deepset/gbert-base for 8 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.227271,0.712329,0.614173,0.659619
2,0.316000,0.182275,0.748936,0.692913,0.719836
3,0.145200,0.207398,0.771318,0.783465,0.777344
4,0.057100,0.246431,0.743295,0.763780,0.753398
5,0.034700,0.263632,0.746377,0.811024,0.777358
6,0.034700,0.258587,0.768939,0.799213,0.783784
7,0.012600,0.275203,0.752768,0.803150,0.777143
8,0.007500,0.273943,0.750929,0.795276,0.772467


Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5798.97 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.91      0.95      0.93        55
    Krankenhaus       0.90      0.78      0.83       117
       Personal       0.70      0.54      0.61        13
 Pflegepersonal       0.89      0.94      0.92        18
anderer Service       0.80      0.57      0.67        35
 mediz. Service       0.88      0.74      0.80        77

      micro avg       0.88      0.77      0.82       315
      macro avg       0.85      0.75      0.79       315
   weighted avg       0.88      0.77      0.82       315

Precision Score: 0.8808664259927798
Recall Score: 0.7746031746031746
F1 Score: 0.8243243243243243
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [9]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=10)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 6346.06 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1035: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-Krankenhaus' 'B-mediz. Service' 'B-Personal' 'B-anderer Service'
 'B-Arzt' 'B-Pflegepersonal' 'O']
{0: 5.983217355710193, 1: 4.959959280624364, 2: 18.317042606516292, 3: 10.546176046176047, 4: 6.8239962651727355, 5: 14.014381591562799, 6: 0.1596439493228484}
Training google-bert/bert-base-german-cased for 10 epochs with random seeds 42, 42



...

['B-Krankenhaus' 'B-mediz. Service' 'B-Personal' 'B-anderer Service'
 'B-Arzt' 'B-Pflegepersonal' 'O']
{0: 5.983217355710193, 1: 4.959959280624364, 2: 18.317042606516292, 3: 10.546176046176047, 4: 6.8239962651727355, 5: 14.014381591562799, 6: 0.1596439493228484}
Training deepset/gbert-base for 10 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.212765,0.789216,0.633858,0.703057
2,0.324100,0.201206,0.802817,0.673228,0.732334
3,0.147400,0.207265,0.787500,0.744094,0.765182
4,0.062800,0.262759,0.814815,0.692913,0.748936
5,0.038500,0.275988,0.739300,0.748031,0.743640
6,0.038500,0.273156,0.795745,0.736220,0.764826
7,0.015600,0.308652,0.750958,0.771654,0.761165
8,0.007800,0.317833,0.772549,0.775591,0.774067
9,0.004800,0.330199,0.765625,0.771654,0.768627
10,0.001700,0.328842,0.767442,0.779528,0.773438


Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5867.04 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.86      0.93      0.89        55
    Krankenhaus       0.92      0.72      0.81       117
       Personal       0.64      0.69      0.67        13
 Pflegepersonal       0.85      0.94      0.89        18
anderer Service       0.78      0.51      0.62        35
 mediz. Service       0.76      0.74      0.75        77

      micro avg       0.84      0.75      0.79       315
      macro avg       0.80      0.76      0.77       315
   weighted avg       0.84      0.75      0.79       315

Precision Score: 0.8368794326241135
Recall Score: 0.7492063492063492
F1 Score: 0.7906197654941373
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [10]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=12)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 6294.67 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1035: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-Krankenhaus' 'B-mediz. Service' 'B-Personal' 'B-anderer Service'
 'B-Arzt' 'B-Pflegepersonal' 'O']
{0: 5.983217355710193, 1: 4.959959280624364, 2: 18.317042606516292, 3: 10.546176046176047, 4: 6.8239962651727355, 5: 14.014381591562799, 6: 0.1596439493228484}
Training google-bert/bert-base-german-cased for 12 epochs with random seeds 42, 42



...

['B-Krankenhaus' 'B-mediz. Service' 'B-Personal' 'B-anderer Service'
 'B-Arzt' 'B-Pflegepersonal' 'O']
{0: 5.983217355710193, 1: 4.959959280624364, 2: 18.317042606516292, 3: 10.546176046176047, 4: 6.8239962651727355, 5: 14.014381591562799, 6: 0.1596439493228484}
Training deepset/gbert-base for 12 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.227517,0.702326,0.594488,0.643923
2,0.329600,0.194439,0.745763,0.692913,0.718367
3,0.150800,0.210067,0.790514,0.787402,0.788955
4,0.056400,0.248432,0.787402,0.787402,0.787402
5,0.032400,0.308927,0.725979,0.803150,0.762617
6,0.032400,0.271390,0.780220,0.838583,0.808349
7,0.013700,0.273768,0.785992,0.795276,0.790607
8,0.006900,0.317414,0.759542,0.783465,0.771318
9,0.003500,0.322511,0.747331,0.826772,0.785047
10,0.001700,0.307005,0.771218,0.822835,0.796190


Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5903.72 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.87      0.95      0.90        55
    Krankenhaus       0.89      0.77      0.83       117
       Personal       0.60      0.69      0.64        13
 Pflegepersonal       1.00      0.94      0.97        18
anderer Service       0.64      0.40      0.49        35
 mediz. Service       0.83      0.78      0.81        77

      micro avg       0.84      0.77      0.80       315
      macro avg       0.80      0.76      0.77       315
   weighted avg       0.84      0.77      0.80       315

Precision Score: 0.8432055749128919
Recall Score: 0.7682539682539683
F1 Score: 0.8039867109634551
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

### k-fold crossvalidation

In [7]:
for model in models:
    print(f'training and results for {model}:')
    ate_model_kfold(data, model, rn1=42, rn2=42, k=3, epochs=5)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting fold 1/3


Map: 100%|██████████| 169/169 [00:00<00:00, 7629.62 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:851: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training fold 1


...

Map: 100%|██████████| 169/169 [00:00<00:00, 6661.88 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:851: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training fold 3


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.002999,0.997674,0.997674,0.997674
2,0.029600,0.006367,0.997674,0.997674,0.997674
3,0.016900,0.006430,0.993056,0.997674,0.995360
4,0.016900,0.007164,0.995360,0.997674,0.996516
5,0.003100,0.008883,0.995360,0.997674,0.996516


Evaluating fold 3


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Fold 3 Results - Precision: 1.0, Recall: 0.9958677685950413, F1: 0.9979296066252588

=== Final Cross-Validation Results ===
Average Precision: 0.9564286786704311
Average Recall: 0.9258725733992055
Average F1 Score: 0.9406108239637483
Average epoch time: 13.19s ± 0.09s
Total training time: 3.3 minutes
Peak memory usage: 3077.0MB
Average batch time: 0.0374s ± 0.0003s
GPUs used: NVIDIA A30
Training complete. Model directory for fold 1 deleted to free memory.
Training complete. Model directory for fold 2 deleted to free memory.
Training complete. Model directory for fold 3 deleted to free memory.



In [8]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model_kfold(data, model, rn1=42, rn2=42, k=3, epochs=5)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting fold 1/3


Map: 100%|██████████| 169/169 [00:00<00:00, 7515.64 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1254: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-mediz. Service' 'B-anderer Service' 'B-Arzt' 'B-Krankenhaus' 'O'
 'B-Personal' 'B-Pflegepersonal']
{0: 4.959959280624364, 1: 10.546176046176047, 2: 6.8239962651727355, 3: 5.983217355710193, 4: 0.1596439493228484, 5: 18.317042606516292, 6: 14.014381591562799}
Training fold 1


...

Map: 100%|██████████| 169/169 [00:00<00:00, 7037.98 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1254: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-mediz. Service' 'B-anderer Service' 'B-Arzt' 'B-Krankenhaus' 'O'
 'B-Personal' 'B-Pflegepersonal']
{0: 4.959959280624364, 1: 10.546176046176047, 2: 6.8239962651727355, 3: 5.983217355710193, 4: 0.1596439493228484, 5: 18.317042606516292, 6: 14.014381591562799}
Training fold 3


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.012702,0.983871,0.993023,0.988426
2,0.055500,0.006445,0.981567,0.990698,0.986111
3,0.022400,0.010198,0.990741,0.995349,0.993039
4,0.022400,0.012780,0.986175,0.995349,0.990741
5,0.005400,0.010794,0.986175,0.995349,0.990741


Evaluating fold 3


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.99      1.00      1.00       122
    Krankenhaus       1.00      1.00      1.00       137
       Personal       1.00      1.00      1.00        17
 Pflegepersonal       1.00      1.00      1.00        33
anderer Service       1.00      1.00      1.00        32
 mediz. Service       0.99      0.99      0.99       143

      micro avg       0.99      1.00      0.99       484
      macro avg       1.00      1.00      1.00       484
   weighted avg       0.99      1.00      0.99       484

Fold 3 Results - Precision: 0.9938144329896907, Recall: 0.9958677685950413, F1: 0.9948400412796697

=== Final Cross-Validation Results ===
Average Precision: 0.9201459416468646
Average Recall: 0.877451984059214
Average F1 Score: 0.8970808132568558
Average epoch time: 12.83s ± 0.09s
Total training time: 3.2 m

### 3. Performance for best models (standard ATE: dbmdz BERT cased, category-aware ATE: GBERT)

In [5]:
for epoch in [5, 6, 7, 8, 10, 12]:
    print(f'training and results for DBMDZ Bert for {epoch} epochs:')
    ate_model(data, "dbmdz/bert-base-german-cased", rn1=42, rn2=42, epochs=epoch)
    print()
# GPU: Tesla V100-PCIE-32GB 

training and results for DBMDZ Bert for 5 epochs:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 3498.46 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for dbmdz/bert-base-german-cased with 5 epochs and random seeds: 42, 42



...

['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for dbmdz/bert-base-german-cased with 12 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.157434,0.807377,0.775591,0.791165
2,0.196500,0.155365,0.843621,0.807087,0.824950
3,0.091300,0.222058,0.825726,0.783465,0.804040
4,0.035200,0.261567,0.795539,0.842520,0.818356
5,0.021500,0.301959,0.807229,0.791339,0.799205
6,0.021500,0.388211,0.786260,0.811024,0.798450
7,0.007600,0.430262,0.781609,0.803150,0.792233
8,0.003000,0.408844,0.807229,0.791339,0.799205
9,0.002200,0.406708,0.818565,0.763780,0.790224
10,0.002100,0.407874,0.795539,0.842520,0.818356


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 2983.81 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.89      0.74      0.81       315

   micro avg       0.89      0.74      0.81       315
   macro avg       0.89      0.74      0.81       315
weighted avg       0.89      0.74      0.81       315

Precision Score: 0.8893129770992366
Recall Score: 0.7396825396825397
F1 Score: 0.807625649913345
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Label

In [ ]:
for epoch in [5, 6, 7, 8, 10, 12]:
    print(f'training and results for DBMDZ Bert for {epoch} epochs:')
    ate_model(data, "dbmdz/bert-base-german-cased", rn1=42, rn2=42, epochs=epoch)
    print()
# GPU: NVIDIA GeForce RTX 2080 Ti   

training and results for DBMDZ Bert for 5 epochs:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 3428.63 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for dbmdz/bert-base-german-cased with 5 epochs and random seeds: 42, 42



...

['B-ASPECT' 'O']
{0: 4.755042290175667, 1: 0.5587538226299694}
Training results for dbmdz/bert-base-german-cased with 12 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.157616,0.807377,0.775591,0.791165
2,0.196500,0.144559,0.846473,0.803150,0.824242
3,0.091400,0.210400,0.826613,0.807087,0.816733
4,0.034300,0.264129,0.801444,0.874016,0.836158
5,0.020800,0.353754,0.805447,0.814961,0.810176
6,0.020800,0.408565,0.803150,0.803150,0.803150
7,0.008000,0.401566,0.789668,0.842520,0.815238
8,0.003900,0.411526,0.799257,0.846457,0.822180
9,0.001600,0.426865,0.801556,0.811024,0.806262
10,0.001300,0.423290,0.800000,0.834646,0.816956


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 3644.21 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.85      0.83      0.84       315

   micro avg       0.85      0.83      0.84       315
   macro avg       0.85      0.83      0.84       315
weighted avg       0.85      0.83      0.84       315

Precision Score: 0.8529411764705882
Recall Score: 0.8285714285714286
F1 Score: 0.8405797101449276
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labe

In [5]:
for epoch in [5, 6, 7, 8, 10, 12]:
    print(f'training and results for DBMDZ Bert for {epoch} epochs:')
    ate_model(data, "dbmdz/bert-base-german-cased", rn1=42, rn2=42, epochs=epoch)
    print()
# GPU: NVIDIA A30   

training and results for DBMDZ Bert for 5 epochs:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 6558.37 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:637: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for dbmdz/bert-base-german-cased with 5 epochs and random seeds: 42, 42



...

['O' 'B-ASPECT']
{0: 0.5587538226299694, 1: 4.755042290175667}
Training results for dbmdz/bert-base-german-cased with 12 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.156757,0.807377,0.775591,0.791165
2,0.196500,0.149944,0.845188,0.795276,0.819473
3,0.090600,0.217388,0.844444,0.748031,0.793319
4,0.035200,0.278473,0.803774,0.838583,0.820809
5,0.017200,0.329946,0.814394,0.846457,0.830116
6,0.017200,0.386355,0.784906,0.818898,0.801541
7,0.008100,0.424874,0.770609,0.846457,0.806754
8,0.003600,0.431949,0.804511,0.842520,0.823077
9,0.001700,0.446138,0.812977,0.838583,0.825581
10,0.000600,0.467103,0.829960,0.807087,0.818363


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5929.25 examples/s]


Unique predicted label IDs: {0, 1}
Expected label IDs: {0, 1}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.89      0.78      0.83       315

   micro avg       0.89      0.78      0.83       315
   macro avg       0.89      0.78      0.83       315
weighted avg       0.89      0.78      0.83       315

Precision Score: 0.8916967509025271
Recall Score: 0.7841269841269841
F1 Score: 0.8344594594594594
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labe

In [6]:
for epoch in [5, 6, 7, 8, 10, 12]:
    print(f'training and results for DBMDZ Bert for {epoch} epochs:')
    ate_cat_model(data, "deepset/gbert-base", rn1=42, rn2=42, epochs=epoch)
    print()
# GPU: Tesla V100-PCIE-32GB

training and results for DBMDZ Bert for 5 epochs:


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', '

Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 3368.73 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1044: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-Arzt' 'B-mediz. Service' 'B-Personal' 'B-Pflegepersonal'
 'B-anderer Service' 'B-Krankenhaus']
{0: 0.1596439493228484, 1: 6.8239962651727355, 2: 4.959959280624364, 3: 18.317042606516292, 4: 14.014381591562799, 5: 10.546176046176047, 6: 5.983217355710193}
Training deepset/gbert-base for 5 epochs with random seeds 42, 42



...

['O' 'B-Arzt' 'B-mediz. Service' 'B-Personal' 'B-Pflegepersonal'
 'B-anderer Service' 'B-Krankenhaus']
{0: 0.1596439493228484, 1: 6.8239962651727355, 2: 4.959959280624364, 3: 18.317042606516292, 4: 14.014381591562799, 5: 10.546176046176047, 6: 5.983217355710193}
Training deepset/gbert-base for 12 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.212552,0.793103,0.633858,0.704595
2,0.324100,0.199718,0.798122,0.669291,0.728051
3,0.147800,0.208771,0.785124,0.748031,0.766129
4,0.063800,0.260315,0.826291,0.692913,0.753747
5,0.039000,0.279518,0.751938,0.763780,0.757812
6,0.039000,0.274944,0.782258,0.763780,0.772908
7,0.017500,0.331395,0.768000,0.755906,0.761905
8,0.008000,0.366493,0.748092,0.771654,0.759690
9,0.003100,0.384498,0.767932,0.716535,0.741344
10,0.002300,0.356591,0.760148,0.811024,0.784762


Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 2652.07 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.93      0.95      0.94        55
    Krankenhaus       0.88      0.70      0.78       117
       Personal       0.75      0.69      0.72        13
 Pflegepersonal       0.85      0.94      0.89        18
anderer Service       0.75      0.51      0.61        35
 mediz. Service       0.73      0.75      0.74        77

      micro avg       0.83      0.75      0.79       315
      macro avg       0.82      0.76      0.78       315
   weighted avg       0.83      0.75      0.78       315

Precision Score: 0.8309859154929577
Recall Score: 0.7492063492063492
F1 Score: 0.7879799666110183
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [5]:
for epoch in [5, 6, 7, 8, 10, 12]:
    print(f'training and results for gBert for {epoch} epochs:')
    ate_cat_model(data, "deepset/gbert-base", rn1=42, rn2=42, epochs=epoch)
    print()
# GPU: NVIDIA GeForce RTX 2080 Ti

training and results for gBert for 5 epochs:


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', '

Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 3759.90 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1044: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-Arzt' 'B-mediz. Service' 'B-anderer Service' 'O' 'B-Pflegepersonal'
 'B-Personal' 'B-Krankenhaus']
{0: 6.8239962651727355, 1: 4.959959280624364, 2: 10.546176046176047, 3: 0.1596439493228484, 4: 14.014381591562799, 5: 18.317042606516292, 6: 5.983217355710193}
Training deepset/gbert-base for 5 epochs with random seeds 42, 42



...

['B-Arzt' 'B-mediz. Service' 'B-anderer Service' 'O' 'B-Pflegepersonal'
 'B-Personal' 'B-Krankenhaus']
{0: 6.8239962651727355, 1: 4.959959280624364, 2: 10.546176046176047, 3: 0.1596439493228484, 4: 14.014381591562799, 5: 18.317042606516292, 6: 5.983217355710193}
Training deepset/gbert-base for 12 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.212973,0.793103,0.633858,0.704595
2,0.324000,0.199952,0.798122,0.669291,0.728051
3,0.147200,0.198577,0.787755,0.759843,0.773547
4,0.070400,0.261719,0.733906,0.673228,0.702259
5,0.039500,0.300803,0.753086,0.720472,0.736419
6,0.039500,0.311564,0.802632,0.720472,0.759336
7,0.013800,0.316394,0.770992,0.795276,0.782946
8,0.007500,0.327303,0.750000,0.755906,0.752941
9,0.003200,0.363462,0.755102,0.728346,0.741483
10,0.002700,0.330890,0.776515,0.807087,0.791506


Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 3555.20 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.93      0.96      0.95        55
    Krankenhaus       0.86      0.74      0.80       117
       Personal       0.69      0.69      0.69        13
 Pflegepersonal       0.89      0.94      0.92        18
anderer Service       0.71      0.57      0.63        35
 mediz. Service       0.76      0.75      0.76        77

      micro avg       0.83      0.77      0.80       315
      macro avg       0.81      0.78      0.79       315
   weighted avg       0.83      0.77      0.80       315

Precision Score: 0.8299319727891157
Recall Score: 0.7746031746031746
F1 Score: 0.8013136288998358
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [6]:
for epoch in [5, 6, 7, 8, 10, 12]:
    print(f'training and results for gBert for {epoch} epochs:')
    ate_cat_model(data, "deepset/gbert-base", rn1=42, rn2=42, epochs=epoch, save = True)
    print()
# GPU: NVIDIA A30

training and results for gBert for 5 epochs:


Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Some weights of BertForTokenClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferen

Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 6581.29 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1044: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-Krankenhaus' 'B-Personal' 'B-mediz. Service' 'B-Pflegepersonal'
 'B-Arzt' 'B-anderer Service' 'O']
{0: 5.983217355710193, 1: 18.317042606516292, 2: 4.959959280624364, 3: 14.014381591562799, 4: 6.8239962651727355, 5: 10.546176046176047, 6: 0.1596439493228484}
Training deepset/gbert-base for 5 epochs with random seeds 42, 42



...

['B-Krankenhaus' 'B-Personal' 'B-mediz. Service' 'B-Pflegepersonal'
 'B-Arzt' 'B-anderer Service' 'O']
{0: 5.983217355710193, 1: 18.317042606516292, 2: 4.959959280624364, 3: 14.014381591562799, 4: 6.8239962651727355, 5: 10.546176046176047, 6: 0.1596439493228484}
Training deepset/gbert-base for 12 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.212159,0.793103,0.633858,0.704595
2,0.324000,0.200212,0.802817,0.673228,0.732334
3,0.147300,0.207513,0.792531,0.751969,0.771717
4,0.063300,0.249735,0.827273,0.716535,0.767932
5,0.037100,0.268996,0.729008,0.751969,0.740310
6,0.037100,0.275405,0.797571,0.775591,0.786427
7,0.017000,0.302324,0.769841,0.763780,0.766798
8,0.007500,0.305909,0.755474,0.814961,0.784091
9,0.004700,0.317125,0.732342,0.775591,0.753346
10,0.001300,0.313503,0.774908,0.826772,0.800000



Best Model saved at: ./saved_models/ate_cat_deepset_gbert-base_42_42_12

Tokenizer for best Model saved at: ./saved_tokenizers/ate_cat_deepset_gbert-base_42_42_12
Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 5884.48 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.95      0.95      0.95        55
    Krankenhaus       0.90      0.70      0.79       117
       Personal       0.75      0.69      0.72        13
 Pflegepersonal       0.81      0.94      0.87        18
anderer Service       0.78      0.51      0.62        35
 mediz. Service       0.74      0.83      0.78        77

      micro avg       0.84      0.77      0.80       315
      macro avg       0.82      0.77      0.79       315
   weighted avg       0.84      0.77      0.80       315

Precision Score: 0.8373702422145328
Recall Score: 0.7682539682539683
F1 Score: 0.8013245033112584
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [7]:
ate_cat_model(data, "deepset/gbert-base", rn1=42, rn2=42, epochs=7, save = True) #v1

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Some weights of BertForTokenClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferen

Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 4241.46 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1044: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-Personal' 'O' 'B-Arzt' 'B-Krankenhaus' 'B-mediz. Service'
 'B-Pflegepersonal' 'B-anderer Service']
{0: 18.317042606516292, 1: 0.1596439493228484, 2: 6.8239962651727355, 3: 5.983217355710193, 4: 4.959959280624364, 5: 14.014381591562799, 6: 10.546176046176047}
Training deepset/gbert-base for 7 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.236440,0.702326,0.594488,0.643923
2,0.316500,0.208870,0.790698,0.669291,0.724947
3,0.139500,0.213957,0.783898,0.728346,0.755102
4,0.055400,0.239321,0.742188,0.748031,0.745098
5,0.032200,0.283362,0.744275,0.767717,0.755814
6,0.032200,0.279220,0.768627,0.771654,0.770138
7,0.011300,0.297911,0.780083,0.740157,0.759596



Best Model saved at: ./saved_models/ate_cat_deepset_gbert-base_42_42_7

Tokenizer for best Model saved at: ./saved_tokenizers/ate_cat_deepset_gbert-base_42_42_7
Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 3922.93 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.87      0.95      0.90        55
    Krankenhaus       0.91      0.73      0.81       117
       Personal       0.69      0.69      0.69        13
 Pflegepersonal       1.00      0.94      0.97        18
anderer Service       0.76      0.54      0.63        35
 mediz. Service       0.83      0.70      0.76        77

      micro avg       0.86      0.75      0.80       315
      macro avg       0.84      0.76      0.80       315
   weighted avg       0.86      0.75      0.80       315

Precision Score: 0.8644688644688645
Recall Score: 0.7492063492063492
F1 Score: 0.8027210884353742
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [5]:
ate_cat_model(data, "deepset/gbert-base", rn1=42, rn2=42, epochs=7, save = True) #v2

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', '

Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 3790.96 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1044: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['O' 'B-Krankenhaus' 'B-Personal' 'B-Pflegepersonal' 'B-mediz. Service'
 'B-Arzt' 'B-anderer Service']
{0: 0.1596439493228484, 1: 5.983217355710193, 2: 18.317042606516292, 3: 14.014381591562799, 4: 4.959959280624364, 5: 6.8239962651727355, 6: 10.546176046176047}
Training deepset/gbert-base for 7 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.224845,0.727273,0.629921,0.675105
2,0.320800,0.215142,0.766816,0.673228,0.716981
3,0.145000,0.240616,0.768340,0.783465,0.775828
4,0.058600,0.298305,0.764192,0.688976,0.724638
5,0.033100,0.304550,0.732394,0.818898,0.773234
6,0.033100,0.298057,0.755474,0.814961,0.784091
7,0.012300,0.310792,0.754941,0.751969,0.753452



Best Model saved at: ./saved_models/ate_cat_deepset_gbert-base_42_42_7

Tokenizer for best Model saved at: ./saved_tokenizers/ate_cat_deepset_gbert-base_42_42_7
Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 3455.84 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.88      0.91      0.89        55
    Krankenhaus       0.91      0.73      0.81       117
       Personal       0.69      0.69      0.69        13
 Pflegepersonal       0.89      0.94      0.92        18
anderer Service       0.81      0.63      0.71        35
 mediz. Service       0.76      0.78      0.77        77

      micro avg       0.84      0.77      0.81       315
      macro avg       0.83      0.78      0.80       315
   weighted avg       0.85      0.77      0.80       315

Precision Score: 0.84375
Recall Score: 0.7714285714285715
F1 Score: 0.8059701492537314
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O'

In [5]:
ate_cat_model(data, "deepset/gbert-base", rn1=42, rn2=42, epochs=7, save = True) # v3

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', '

Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 6698.15 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1044: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-Personal' 'B-Arzt' 'B-mediz. Service' 'O' 'B-Krankenhaus'
 'B-Pflegepersonal' 'B-anderer Service']
{0: 18.317042606516292, 1: 6.8239962651727355, 2: 4.959959280624364, 3: 0.1596439493228484, 4: 5.983217355710193, 5: 14.014381591562799, 6: 10.546176046176047}
Training deepset/gbert-base for 7 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.220122,0.761468,0.653543,0.703390
2,0.334400,0.203986,0.747863,0.688976,0.717213
3,0.145100,0.242743,0.769547,0.736220,0.752515
4,0.061600,0.275232,0.800905,0.696850,0.745263
5,0.035900,0.290093,0.741573,0.779528,0.760077
6,0.035900,0.272546,0.792683,0.767717,0.780000
7,0.014700,0.277809,0.768924,0.759843,0.764356



Best Model saved at: ./saved_models/ate_cat_deepset_gbert-base_42_42_7

Tokenizer for best Model saved at: ./saved_tokenizers/ate_cat_deepset_gbert-base_42_42_7
Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 6173.43 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.91      0.96      0.94        55
    Krankenhaus       0.94      0.70      0.80       117
       Personal       0.75      0.69      0.72        13
 Pflegepersonal       1.00      0.94      0.97        18
anderer Service       0.88      0.43      0.58        35
 mediz. Service       0.86      0.74      0.80        77

      micro avg       0.91      0.74      0.81       315
      macro avg       0.89      0.75      0.80       315
   weighted avg       0.91      0.74      0.81       315

Precision Score: 0.9066147859922179
Recall Score: 0.7396825396825397
F1 Score: 0.8146853146853148
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O

In [5]:
ate_cat_model(data, "deepset/gbert-base", rn1=42, rn2=42, epochs=7, save = True) # v4

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', '

Mapping the data


Map: 100%|██████████| 101/101 [00:00<00:00, 4461.04 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA_German_Hospital_Reviews/functions/ate_model_train_OB.py:1044: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


['B-anderer Service' 'B-Krankenhaus' 'B-Arzt' 'B-mediz. Service'
 'B-Pflegepersonal' 'O' 'B-Personal']
{0: 10.546176046176047, 1: 5.983217355710193, 2: 6.8239962651727355, 3: 4.959959280624364, 4: 14.014381591562799, 5: 0.1596439493228484, 6: 18.317042606516292}
Training deepset/gbert-base for 7 epochs with random seeds 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.244711,0.721951,0.582677,0.644880
2,0.328600,0.194730,0.764706,0.716535,0.739837
3,0.142800,0.222282,0.741935,0.724409,0.733068
4,0.056000,0.225319,0.802372,0.799213,0.800789
5,0.033500,0.262696,0.739623,0.771654,0.755299
6,0.033500,0.279393,0.759542,0.783465,0.771318
7,0.011700,0.301292,0.740310,0.751969,0.746094



Best Model saved at: ./saved_models/ate_cat_deepset_gbert-base_42_42_7

Tokenizer for best Model saved at: ./saved_tokenizers/ate_cat_deepset_gbert-base_42_42_7
Evaluating on test data



Map: 100%|██████████| 102/102 [00:00<00:00, 3829.66 examples/s]


Unique predicted label IDs: {0, 2, 3, 4, 5, 6, 7}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.93      0.91      0.92        55
    Krankenhaus       0.92      0.77      0.84       117
       Personal       0.75      0.69      0.72        13
 Pflegepersonal       1.00      0.83      0.91        18
anderer Service       0.60      0.43      0.50        35
 mediz. Service       0.81      0.81      0.81        77

      micro avg       0.86      0.77      0.81       315
      macro avg       0.83      0.74      0.78       315
   weighted avg       0.85      0.77      0.81       315

Precision Score: 0.8576512455516014
Recall Score: 0.765079365079365
F1 Score: 0.8087248322147651
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O'